In [ ]:
#importing the necessay libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
#importing the data
data_train = pd.read_csv('nn groupassignment/train.csv', parse_dates = ['Date'], dayfirst=True, low_memory=False )
data_test = pd.read_csv('nn groupassignment/test.csv', parse_dates = ['Date'], dayfirst=True, low_memory=False)
data_store = data_test = pd.read_csv('nn groupassignment/store.csv', low_memory=False)

In [ ]:
data_train.head() #printing the first five lines after importing

In [ ]:
data_store.head()

In [ ]:
data_test.head()

In [ ]:
# we will merge the training and test data with the store data
new_train = pd.merge(left =data_store,right= data_train, left_on = 'Store', right_on = 'Store', how = 'inner')
new_test = pd.merge(left=data_store,right= data_test, left_on = 'Store', right_on = 'Store', how = 'inner')


In [ ]:
new_train.head()

In [ ]:
new_train.dtypes # checking the data types

In [ ]:
#finding the null cells 
new_train.isnull().sum()

In [ ]:
#filling the null cells
pd.Series.replace
new_train['CompetitionDistance'] = new_train['CompetitionDistance'].replace('nan',np.nan).fillna(0)
new_train['Promo2SinceWeek'] = new_train['Promo2SinceWeek'].replace('nan',np.nan).fillna(0)
new_train['Promo2SinceYear'] = new_train['Promo2SinceYear'].replace('nan',np.nan).fillna(0)
new_train['CompetitionOpenSinceMonth'] = new_train['CompetitionOpenSinceMonth'].replace('nan',np.nan).fillna(0)
new_train['CompetitionOpenSinceYear'] = new_train['CompetitionOpenSinceYear'].replace('nan',np.nan).fillna(0)
new_train['PromoInterval'] = new_train['PromoInterval'].replace('nan',np.nan).fillna(0)
new_train.isnull().sum()

In [ ]:
new_train.head()

In [ ]:
#converting the year values, competition open since month, competition open since year,promo open since month, promo open since year
new_train['Promo2SinceWeek'] = new_train['Promo2SinceWeek'].astype('int64')
new_train['Promo2SinceYear'] = new_train['Promo2SinceYear'].astype('int64')
new_train['CompetitionOpenSinceYear'] = new_train['CompetitionOpenSinceYear'].astype('int64')
new_train['CompetitionOpenSinceMonth'] = new_train['CompetitionOpenSinceMonth'].astype('int64') 



In [ ]:
new_train.dtypes

In [ ]:
time0 = pd.Timestamp('2012-01-01 00:00:00') # coverting the time into timestamp
new_train['Date'] = (new_train['Date'] - time0).values.astype(float)
new_train.head()

In [ ]:
new_train = pd.get_dummies(new_train, columns=['StoreType']) #converting the StoreType data (categorical)
new_train.head()

In [ ]:
new_train = pd.get_dummies(new_train, columns=['Assortment']) #converting the Assortment data(categorical)
new_train.head()

In [ ]:
new_train = pd.get_dummies(new_train, columns=['StateHoliday'])#converting the Stateholiday data(numerical)
new_train.head()

In [ ]:
new_train = pd.get_dummies(new_train, columns=['PromoInterval'])
new_train.head()

new_train.dtypes

In [ ]:
new_train.dtypes

In [ ]:
usable_cols1 = list(new_train)[:8] #creating a training set

In [ ]:
usable_cols1

In [ ]:
usable_cols2 = list(new_train)[9:]

In [ ]:
usable_cols2

In [ ]:
usable_cols = usable_cols1 + usable_cols2

In [ ]:
usable_cols

In [ ]:
training_data = new_train[usable_cols]

In [ ]:
training_data.dtypes

In [ ]:
training_data = new_train[usable_cols].astype(float)
training_data

In [ ]:
training_data.info()

In [ ]:
training_data.shape

In [ ]:
scaled = StandardScaler()  # scaling the data

In [ ]:
scaled = scaled.fit(training_data)

In [ ]:
training_data_scaled = scaled.transform(training_data)

In [ ]:
training_data_scaled

In [ ]:
training_x = []
training_y = []
future_days = 1
past_days = 14

In [ ]:
training_data.tail(2000).plot.line()

In [ ]:
for i in range(past_days, len(training_data_scaled) - future_days +1):     # splitting the data into training and target
    training_x.append(training_data_scaled[i - past_days:i, 0:training_data.shape[1]])
    training_y.append(training_data_scaled[i + future_days -1:i + future_days, 8])

In [ ]:
training_x

In [ ]:
training_x, training_y = np.array(training_x), np.array(training_y)
print(training_x.shape, training_y.shape)

In [ ]:
model = Sequential() #developing the LSTM model for training
model.add(LSTM(100,activation='relu',input_shape=(training_x.shape[1],training_x.shape[2]),return_sequences=True))
model.add(LSTM(100,activation='relu',return_sequences=True))
# model.add(Dropout(0.2))
model.add(Dense(training_y.shape[1]))
model.compile(loss='mean_squared_error',optimizer='adam')


In [ ]:
model.summary()

In [ ]:
history = model.fit(training_x,training_y, epochs=5, batch_size=161, verbose=1)  #fitting the model 

In [ ]:
#predicting the future sales
future_days = 42
forecast_period_dates = pd.date_range(list(new_train['Date'])[-1], preriods=future_days, freq='7d')
forcast = model.predict(training_x[-future_days:])
predictions = np.repeat(forecast,training_data.shape[1], axis=-1)


In [ ]:

new_future_d = scaler.inverse_transform(predictions)[:,8] #denomalising the data